In [1]:
# %%
# DO K-means and generate output diagrams

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from sqlalchemy import create_engine, text

import mpl_toolkits.mplot3d
import numpy as np

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# List of columns that will be input to kmeans
used_columns = []

def run():
    data = query_db()
    do_kmean(data)
    

def do_kmean(data):
    
    kmeans = KMeans(n_clusters=8, random_state=0, n_init="auto").fit(data)
    X = data
    y = kmeans.labels_

    # Definition of enumerators runs the kmeans algos
    estimators = [
        ("k_means_iris_8", KMeans(n_clusters=8, custom=False, alpha=1, dimentions=2)),
        ("k_means_iris_3", KMeans(n_clusters=3, custom=False, alpha=1, dimentions=2)),
        ("k_means_iris_20", KMeans(n_clusters=20, custom=False, alpha=1, dimentions=2))

    ]

    fig = plt.figure(figsize=(20, 16))
    titles = ["8 clusters", "3 clusters", "20 clusters"]
    for idx, ((name, est), title) in enumerate(zip(estimators, titles)):
        ax = fig.add_subplot(2, 2, idx + 1, projection="3d", elev=48, azim=134)
        est.fit(X)
        labels = est.labels_

        ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=labels.astype(float), edgecolor="k")

        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])
        ax.zaxis.set_ticklabels([])
        ax.set_xlabel("Long")
        ax.set_ylabel("Lat")
        ax.invert_yaxis()
        ax.set_zlabel("DOY")
        ax.set_title(title)
            
        
    plt.subplots_adjust(wspace=0.25, hspace=0.25)
    plt.show()

    print("estimator 0 " + str(estimators[0]))
    if True: #for (name, est) in estimators[0][1]:
        plt.figure(figsize=(20, 8))
        title = "8 clusters"
        # Print out the value of the clusters for the 8 cluster example
        if(title == "8 clusters"):
            #clusterDiagram = fig.add_subplot() # create a new subplot for info about the clusters

            clusterID = 1 # identify each cluster 
            for cluster in estimators[0][1].cluster_centers_: # get cluster centers to represent the cluster
                #print("Cluster " + str(clusterID) + " :")
                #for val_index in range(0,len(cluster)): # displays all the clusters values for each column
                    #print(str(used_columns[val_index]) + ": " + str(cluster[val_index]))
                
                # plot the current clusters values in the diagram
                plt.bar(used_columns, cluster, label = "Cluster " + str(clusterID))
                clusterID = clusterID + 1

            plt.xticks(rotation=90)
            plt.legend()
        
    plt.show()
            
#query db, return array
def query_db():
    engine = create_engine("sqlite:///updated_fires_db.sqlite")

    # NWCG_REPORTING_AGENCY
    # FIRE_YEAR
    # DISCOVERY_DATE
    # DISCOVERY_DOY
    # DISCOVERY_TIME
    # NWCG_CAUSE_CLASSIFICATION
    # NWCG_GENERAL_CAUSE
    # FIRE_SIZE
    # FIRE_SIZE_CLASS
    # LATITUDE
    # LONGITUDE
    # STATE
    # COUNTY

    query = '''
        SELECT REAL_LATITUDE, REAL_LONGITUDE, REAL_DISCOVERY_DOY, REAL_FIRE_SIZE FROM Fires 
        LIMIT 5000
    '''
    
    with engine.connect() as connection:
        result = connection.execute(text(
        query
            )
        )

        # Gets all column names
        table_query = "PRAGMA table_info(Fires);"
        existing_columns = [] # list of all colmns in the fire table
        result = connection.execute(text(table_query))
        data = []
        for row in result:
            a = []
            for item in row:
                a.append(item)

            data.append(a)
        # print(data)
        column_infos = np.array(data)

        # For each column we get several parts of info that we must proccess
        for column_info in column_infos:
            
            column_info_parts = str(column_info).split(" ") # seperate all these parts into individual pieces
            existing_columns.append(column_info_parts[1].strip("'")) # remove quotes from the column name part and add it

        #print(str(existing_columns))
        
        used_columns.extend(["REAL_LATITUDE", "REAL_LONGITUDE", "REAL_DISCOVERY_DOY", "REAL_FIRE_SIZE", "REAL_FIRE_YEAR"])
        query = "SELECT " # "REAL_" columns need to be added manually as they are what we use to display and order matters 

        # Adds the explicit "REAL_" columns to the query
        first_col = True
        for col in used_columns:
            # only adds commas after the first col
            if(not first_col):
                query += ", "
            query += col
            first_col = False
            
        # Add all columns to list of things to group on
        for col_name in existing_columns:
            if(col_name != 'OBJECTID' and 'STATE_' not in col_name and (col_name.startswith("AGENCY_") or col_name.startswith("CAUSE_"))): # Dont group on the primary key OBJECTID
                query += ", "  + col_name
                used_columns.append(col_name)
        query += " FROM Fires"
        query += ""
        query += ";"

        print("query: " + query)

        result = connection.execute(text(
        query
            )
        )
        
        data = []
        for row in result:
            a = []
            for item in row:
                a.append(item)

            data.append(a)
            # print(data)
        x = np.array(data)
        
        # print(x)
    return x


    
if __name__ == '__main__':
    run()
    #data = query_db()
    #print(data)

# %%


query: SELECT REAL_LATITUDE, REAL_LONGITUDE, REAL_DISCOVERY_DOY, REAL_FIRE_SIZE, REAL_FIRE_YEAR, AGENCY_FS, AGENCY_BIA, AGENCY_TRIBE, AGENCY_BLM, AGENCY_NPS, AGENCY_BOR, AGENCY_FWS, AGENCY_ST_C_L, AGENCY_DOD, AGENCY_IA, AGENCY_DOE, CAUSE_POWER_GENERATION_TRANSMISSION_DISTRIBUTION, CAUSE_NATURAL, CAUSE_DEBRIS_AND_OPEN_BURNING, CAUSE_MISSING_DATA_NOT_SPECIFIED_UNDETERMINED, CAUSE_RECREATION_AND_CEREMONY, CAUSE_EQUIPMENT_AND_VEHICLE_USE, CAUSE_ARSON_INCENDIARISM, CAUSE_FIREWORKS, CAUSE_OTHER_CAUSES, CAUSE_RAILROAD_OPERATIONS_AND_MAINTENANCE, CAUSE_SMOKING, CAUSE_MISUSE_OF_FIRE_BY_A_MINOR, CAUSE_FIREARMS_AND_EXPLOSIVES_USE FROM Fires;


TypeError: KMeans.__init__() got an unexpected keyword argument 'dimentions'

In [3]:
#import pandas as pd
import numpy as np
#import geopandas as gpd
#import matplotlib.pyplot as plt
#from shapely.geometry import Point, Polygon
from sqlalchemy import create_engine, text

#import mpl_toolkits.mplot3d
import numpy as np

def test_values_db():
    engine = create_engine("sqlite:///updated_fires_db.sqlite")

    # NWCG_REPORTING_AGENCY
    # FIRE_YEAR
    # DISCOVERY_DATE
    # DISCOVERY_DOY
    # DISCOVERY_TIME
    # NWCG_CAUSE_CLASSIFICATION
    # NWCG_GENERAL_CAUSE
    # FIRE_SIZE
    # FIRE_SIZE_CLASS
    # LATITUDE
    # LONGITUDE
    # STATE
    # COUNTY

    query = '''
        SELECT REAL_LATITUDE, REAL_LONGITUDE, REAL_DISCOVERY_DOY, REAL_FIRE_SIZE, CAUSE_NATURAL, CAUSE_FIREWORKS FROM Fires 
    '''
    query = "SELECT * from CALENDAR LIMIT 366"
    with engine.connect() as connection:
        result = connection.execute(text(query))
        data = []
        for row in result:
            a = []
            for item in row:
                a.append(item)

            data.append(a)
        # print(data)
        result = np.array(data)
        for row in result:
            print(row)
        '''
        lat = []
        long = []
        doy = []
        size = []
        c_nat = []
        c_fireworks = []
        for res in result:
            lat.append(res[0])
            long.append(res[1])
            doy.append(res[2])
            size.append(res[3])
            c_nat.append(res[4])
            c_fireworks.append(res[5])

        print("lat avg: " + str(sum(lat)/len(lat)))
        print("long avg: " + str(sum(long)/len(long)))
        print("doy avg: " + str(sum(doy)/len(doy)))
        print("size avg: " + str(sum(size)/len(size)))
        print("cause natural avg: " + str(sum(c_nat)/len(c_nat)))
        print("cuase natrural max: " + str(max(c_nat)))
        print("cause c_fireworks avg: " + str(sum(c_fireworks)/len(c_fireworks)))
        print("cuase c_fireworks max: " + str(max(c_fireworks)))
        '''
test_values_db()

['1992-01-01' 1 1992 "New Year's Day" 0.02417909842266564]
['1992-01-02' 2 1992 None 0.01208954921133282]
['1992-01-03' 3 1992 None 0.00604477460566641]
['1992-01-04' 4 1992 None 0.003022387302833205]
['1992-01-05' 5 1992 None 0.0015111936514166026]
['1992-01-06' 6 1992 None 0.0007555968257083013]
['1992-01-07' 7 1992 None 0.00037779841285415065]
['1992-01-08' 8 1992 None 0.00018889920642707533]
['1992-01-09' 9 1992 None 9.444960321353766e-05]
['1992-01-10' 10 1992 None 4.722480160676883e-05]
['1992-01-11' 11 1992 None 2.36838575419841e-05]
['1992-01-12' 12 1992 None 1.1949113878891574e-05]
['1992-01-13' 13 1992 None 6.188927155244836e-06]
['1992-01-14' 14 1992 None 3.523204009220516e-06]
['1992-01-15' 15 1992 None 2.619082867806454e-06]
['1992-01-16' 16 1992 None 3.0245031602956196e-06]
['1992-01-17' 17 1992 None 4.942175032932596e-06]
['1992-01-18' 18 1992 None 9.330934422035869e-06]
['1992-01-19' 19 1992 None 1.8385161022157076e-05]
['1992-01-20' 20 1992 'Martin Luther King Jr. Day'